In [11]:
import warnings
warnings.filterwarnings("ignore", message="networkx backend defined more than once")
import sys
import os
import gc
import csv
import copy
import json
import math
import shutil
import random
import pickle
import itertools
import matplotlib
matplotlib.use('Agg')  # 非交互式后端
import numpy as np
import networkx as nx
import tqdm as tqdm
import matplotlib.pyplot as plt
from pathlib import Path
from itertools import product
from protocols import MPC_protocol, MPG_protocol, SP_protocol
from graph import network, set_p_edge

from joblib import Parallel, delayed
from networkx.algorithms.community import greedy_modularity_communities
from networkx.drawing.layout import *

# 获取当前 Notebook 的绝对路径
notebook_path = os.path.abspath("")

from config import DATA_PATHS



In [12]:
mkr = ['x','+','d','o','1','2']+['x']*100
dashs = ['-.','--',':','-']+['-']*100
cols = ['gray','g','b','orange','r','k','purple']+['k']*100
linewidth = 2.2
mks = 5.5
fontsize = 14
sys.path.append("..")  # 确保根目录在 Python 路径中
root_path = DATA_PATHS["input_graphs"]
LOOP_STATE_PATH = "loop_state.pkl"
TEMP_DATA_PATH = "temp_data.pkl"

Find the ER for the MPC, MPG, and SP protocols

In [13]:
def load_data(filepath):
    pos = {}
    user = []

    # Step 1: 读取 JSON 文件
    with open(filepath, "r") as f:
        data = json.load(f)

    # Step 2: 初始化图
    G = nx.Graph()

    # Step 3: 添加节点

    for node in data["nodes"]:
        node_id = node["id"]
        x, y = node["latitude"], node["longitude"]
        G.add_node(node_id, location=node["location"], country=node["country"])  # 添加节点到图
        pos[node_id] = (y, x)  # 保存节点位置，注意 (longitude, latitude)

    # Step 4: 添加边
    for edge in data["links"]:
        source = int(edge["source"])
        target = int(edge["target"])
        G.add_edge(source, target, length=edge["length"])  # 添加边到图

    degree_dict = dict(G.degree())
    degree_items = list(degree_dict.items())
    first_node,first_degree = degree_items[0]
    print(f"First node ID: {first_node}, Degree: {first_degree}")

    user.append(data["nodes"][0]["id"])

    return G,user,pos

In [14]:
def multi_iterative_score_partition_with_drawing(
    G, 
    fixed_node, 
    alpha=1.0, 
    beta=1.0,
    max_rounds=10,
    shuffle_nodes=True,
    pos=None,
    output_path=None
):
    """
    多轮迭代版“打分 + 搬家”分社区并绘制图：
      - 社区数 = degree(fixed_node) + 1
      - 初始: 0 号社区放 fixed_node，其余每个邻居一个社区
      - 分配时: score = alpha * distance + beta * community_size
      - 多轮: 如果搬家能让节点的 score 更低, 就搬家, 直到稳定或 max_rounds
      - 绘图可选。

    同时返回：
      - 最终的社区划分 communities
      - 对所有社区(除了 0 号) 进行“从每个社区选一个节点”的所有组合 all_key_nodes_combos，
        并在每个组合前面加上固定节点 fixed_node。
    """
    # ============ 1) 初始化社区容器 ============
    neighbors = list(G.neighbors(fixed_node))
    num_communities = len(neighbors)  # 除 0 号社区外的社区数
    communities = [set() for _ in range(num_communities + 1)]
    visited = set()

    # 第 0 号社区放 fixed_node
    communities[0].add(fixed_node)
    visited.add(fixed_node)

    # 其余邻居各一个社区
    for i, nb in enumerate(neighbors, start=1):
        communities[i].add(nb)
        visited.add(nb)

    # ============ 2) 初次分配剩余节点 ============
    for node in G.nodes():
        if node not in visited:
            best_score = float('inf')
            best_index = None
            # 尝试放进 1~num_communities 各个社区，看 score 哪个最小
            for i, nb in enumerate(neighbors, start=1):
                dist = nx.shortest_path_length(G, source=node, target=nb)
                size = len(communities[i])
                score = alpha * dist + beta * size
                if score < best_score:
                    best_score = score
                    best_index = i
            communities[best_index].add(node)
            visited.add(node)

    # ============ 3) 多轮迭代搬家 ============
    round_num = 0
    while round_num < max_rounds:
        round_num += 1
        moved_count = 0

        # 获取所有节点（除 fixed_node 外）
        all_nodes = [n for n in G.nodes() if n != fixed_node]

        if shuffle_nodes:
            random.shuffle(all_nodes)

        for node in all_nodes:
            # 找到当前所在社区
            current_idx = None
            for i, comm in enumerate(communities):
                if node in comm:
                    current_idx = i
                    break
            
            # 如果在 0 号社区，则说明节点是 fixed_node，不搬家
            if current_idx == 0:
                continue

            # 当前社区分值
            nb_current = neighbors[current_idx - 1]  # 对应邻居
            dist_current = nx.shortest_path_length(G, source=node, target=nb_current)
            size_current = len(communities[current_idx])
            current_score = alpha * dist_current + beta * size_current

            # 尝试搬去别的社区
            best_score = current_score
            best_index = current_idx

            for i, nb in enumerate(neighbors, start=1):
                if i == current_idx:
                    continue
                dist = nx.shortest_path_length(G, source=node, target=nb)
                size = len(communities[i])
                score = alpha * dist + beta * size

                if score < best_score:
                    best_score = score
                    best_index = i

            # 如果找到更好的社区 => 搬家
            if best_index != current_idx:
                communities[current_idx].remove(node)
                communities[best_index].add(node)
                moved_count += 1

        # 如果没人搬家 => 收敛 => 停止
        if moved_count == 0:
            break

    # ============ 4) 计算所有可能的 key_nodes 组合 ============
    # 注意：0 号社区只包含 fixed_node，它是永远固定的，不做枚举
    #       其余社区(1..num_communities)从中各自选 1 个节点，形成一个组合
    all_key_nodes_combos = []

    # 如果有社区是空的，那么不可能从该社区选出节点
    if all(len(communities[i]) > 0 for i in range(1, num_communities + 1)):
        # itertools.product(...) 会返回所有从各社区选1个节点的组合
        # communities[i] 本身是一个 set，不影响 product 的用法
        # 例如 [setA, setB, setC] => product(setA, setB, setC) 会返回 (a, b, c) 等所有组合
        all_products = product(*(communities[i] for i in range(1, num_communities + 1)))

        # 为每个组合都把 fixed_node(第0社区) 放在开头
        for combo in all_products:
            combo_list = [fixed_node] + list(combo)
            all_key_nodes_combos.append(combo_list)
    else:
        # 如果有社区为空，那么没有可行的枚举
        all_key_nodes_combos = []
    # ============ 5) 绘制社区图 ============
    if pos is None:
        pos = nx.spring_layout(G, seed=42)  # 默认布局

    colors = ["red", "blue", "green", "orange", "purple", "cyan", "yellow", "pink"]
    
    plt.figure(figsize=(8, 6))
    
    # 逐个社区画节点
    for i, community in enumerate(communities):
        nx.draw_networkx_nodes(
            G, pos, nodelist=community,
            node_color=colors[i % len(colors)], 
            label=f"Community {i}",
            alpha=0.8,
            node_size=100
        )

    # 特别标注固定节点（用红色圆形)
    nx.draw_networkx_nodes(
        G, pos, nodelist=[fixed_node],
        node_color="red", node_shape="o",
        node_size=100, alpha=0.9, label="Fixed Node"
    )

    # 绘制边
    nx.draw_networkx_edges(G, pos, edge_color="gray", alpha=0.5)
    # 绘制标签
    nx.draw_networkx_labels(G, pos, font_size=8, font_color="black")

    plt.legend(
        fontsize=6,  
        borderaxespad=0.5, 
        labelspacing=0.2,   
        loc="upper left",
        bbox_to_anchor=(1.05, 1),
    )
    plt.title("Graph with Colored Communities (no farthest-node highlight)")
    plt.tight_layout(pad=2.0)

    if output_path:
        plt.savefig(output_path, dpi=300, bbox_inches='tight')
        print(f"Graph saved to {output_path}")
    else:
        plt.show()
    plt.close()

    # 返回：最终社区划分，以及所有 key_nodes 组合
    return communities, all_key_nodes_combos


In [15]:
# ============== 2) 全局参数 ==============
funcs = [MPC_protocol, MPG_protocol, SP_protocol]
p_range = np.linspace(1, 0.2, 50)

timesteps = 100
reps = 200
alpha = 1.4
beta = 0.105
max_rounds = 10
shuffle_nodes = True


sr_results = []  # 每个文件的 SR 结果都会追加到这里

# 分段退出：针对“组合”数量
chunk_size = 300  
state_file = "loop_state.pkl"

# ============== 3) 读取/初始化进度 ==============
try:
    with open(state_file, "rb") as f:
        progress = pickle.load(f)
    print("恢复进度：", progress)
except FileNotFoundError:
    progress = {
        "subfolder_idx": 0,   # 当前处理到第几个子文件夹
        "file_idx": 0,        # 当前子文件夹内处理到第几个文件
        "combo_idx": 0,       # 当前文件内处理到第几个组合
        "global_combo_count": 0  # 全局已处理的组合数
    }
    print("未发现进度文件，从头开始。")

# 子文件夹列表
subfolders = [sf for sf in root_path.iterdir() if sf.is_dir()]
subfolders.sort()

# ============== 4) 主循环 ==============
for s_idx in range(progress["subfolder_idx"], len(subfolders)):
    subfolder = subfolders[s_idx]
    if not subfolder.is_dir():
        continue

    print(f"Processing subfolder: {subfolder}")

    # 遍历文件
    files = [f for f in subfolder.iterdir() if f.is_file()]
    files.sort()

    for f_idx in range(progress["file_idx"], len(files)):
        file = files[f_idx]
        if not file.is_file():
            continue

        print(f"  Processing file: {file} ...")

        # 每个文件：初始化 counters
        failure_counts = {func.__name__: 0 for func in funcs}
        combination_counter = 0

        # 加载图
        G, users, pos = load_data(file)
        # nx.draw(G, pos, with_labels=True, node_color="lightblue", edge_color="gray")
        # plt.close()

        G = network(G)

        # 输出路径
        class_folder = subfolder.name
        file_path = file.with_suffix(".png")
        file_name = file_path.name

        er_folder_path = Path.cwd().parent.joinpath("new_result", class_folder)
        er_folder_path.mkdir(exist_ok=True)
        er_topology_folder_path = er_folder_path.joinpath(file_name)
        er_topology_folder_path.mkdir(exist_ok=True)
        communities_output_path = Path.cwd().parent.joinpath("communities", class_folder, file_name)

        communities, users_node_combination = multi_iterative_score_partition_with_drawing(
            G, users[0], alpha, beta, max_rounds, shuffle_nodes, pos, communities_output_path
        )

        # sampled_combinations = np.random.choice(
        #     len(users_node_combination),
        #     size=min(100, len(users_node_combination)),
        #     replace=False
        # )


        # 遍历本文件100个组合
        # for sampled_idx, original_idx in enumerate(sampled_combinations):
        #     combo = users_node_combination[original_idx]
        #     combination_counter += 1
        #     progress["global_combo_count"] += 1


        #     # =========== 新增：组合详细信息 ===========
        #     combination_sr = {
        #         "Combination_ID": f"combo_{sampled_idx}",
        #         "Nodes": str(combo),  # 将节点列表转为字符串，如 "[2,5,9]"
        #     }

        #     # 计算 ER
        #     ER = np.zeros((len(funcs), len(p_range)))
        #     results = Parallel(n_jobs=-1, verbose=10)(
        #         delayed(process_single_p)(G, combo, p, funcs, timesteps, reps)
        #         for p in p_range
        #     )

        #     # 将结果填充到 ER 矩阵
        #     for i, p_ers in enumerate(results):
        #         ER[:, i] = p_ers

        #     plot_er_vs_p(p_range, ER, funcs, cols, er_topology_folder_path.joinpath(f'result_for_{str(combo)}'))


        #     # =========== 计算成功率比例 ===========
        #     for func_idx, func in enumerate(funcs):
        #         protocol_er = ER[func_idx, :]
        #         zero_count = np.sum(protocol_er < 1e-10)
        #         success_ratio = 1 - (zero_count / len(p_range))  # 计算成功率比例
        #         combination_sr[func.__name__] = round(success_ratio, 3)  # 保留3位小数
        #     del results, ER
        #     gc.collect() 

        #     # 分段退出：每处理 chunk_size 个组合后退出
        #     if progress["global_combo_count"] % chunk_size == 0:
        #         print(f"已处理 {progress['global_combo_count']} 个组合，准备退出。")
        #         progress["subfolder_idx"] = s_idx
        #         progress["file_idx"] = f_idx
        #         progress["combo_idx"] = sampled_idx + 1  # 下次从下一个组合继续
        #         with open(state_file, "wb") as pf:
        #             pickle.dump(progress, pf)
        #         exit()

        #     # 每个组合处理完更新进度
        #     progress["combo_idx"] = sampled_idx + 1
        #     progress["subfolder_idx"] = s_idx
        #     progress["file_idx"] = f_idx
        #     with open(state_file, "wb") as pf:
        #         pickle.dump(progress, pf)

        #     # =========== 写入CSV文件 ===========
        #     output_subfolder_csv_path = er_topology_folder_path.joinpath(f"{file.stem}_sr_details.csv")

        #     # 定义字段（首次运行时写入表头）
        #     fieldnames = ["Combination_ID", "Nodes"] + [func.__name__ for func in funcs]

        #     # 判断是否需要写表头
        #     write_header = not output_subfolder_csv_path.exists()

        #     with open(output_subfolder_csv_path, mode="a", newline="") as subfile:
        #         csv_writer = csv.DictWriter(subfile, fieldnames=fieldnames)
        #         if write_header:
        #             csv_writer.writeheader()
        #         csv_writer.writerow(combination_sr)

        # # ===========【重点】在文件的所有组合结束后，计算该文件的SR===========
        # sr_for_protocols = {}
        # if combination_counter > 0:
        #     for protocol_name, failures in failure_counts.items():
        #         sr_for_protocols[protocol_name] = (combination_counter - failures) / combination_counter
        # else:
        #     for protocol_name in failure_counts:
        #         sr_for_protocols[protocol_name] = 0

        # # 添加元信息（子文件夹、文件名）
        # sr_for_protocols["Subfolder"] = subfolder.name
        # sr_for_protocols["File"] = file.name
        # sr_results.append(sr_for_protocols)

        # # 这个文件处理完 => 重置 combo_idx，并 file_idx+1
        # progress["combo_idx"] = 0
        # progress["file_idx"] = f_idx + 1
        # with open(state_file, "wb") as pf:
        #     pickle.dump(progress, pf)


未发现进度文件，从头开始。
Processing subfolder: /home/zceeag0/quantum_repeaters_testing/graphs_json/class_0
  Processing file: /home/zceeag0/quantum_repeaters_testing/graphs_json/class_0/TOP_104_USA100.json ...
First node ID: 39, Degree: 6
Graph saved to /home/zceeag0/quantum_repeaters_testing/communities/class_0/TOP_104_USA100.png
  Processing file: /home/zceeag0/quantum_repeaters_testing/graphs_json/class_0/TOP_12_CANARIE.json ...
First node ID: 8, Degree: 5
Graph saved to /home/zceeag0/quantum_repeaters_testing/communities/class_0/TOP_12_CANARIE.png
  Processing file: /home/zceeag0/quantum_repeaters_testing/graphs_json/class_0/TOP_19_CONUS6079.json ...
First node ID: 25, Degree: 2
Graph saved to /home/zceeag0/quantum_repeaters_testing/communities/class_0/TOP_19_CONUS6079.png
  Processing file: /home/zceeag0/quantum_repeaters_testing/graphs_json/class_0/TOP_1_ABILENE.json ...
First node ID: 6, Degree: 3
Graph saved to /home/zceeag0/quantum_repeaters_testing/communities/class_0/TOP_1_ABILENE.png
